In [1]:
from warnings import filterwarnings 
filterwarnings("ignore")

import pandas as pd
import numpy as np

In [2]:
realtor_data_df = pd.read_csv('NewYork-realtor-data.csv')
realtor_data_df["prev_sold_year"] = pd.to_datetime(realtor_data_df["prev_sold_date"], format='%Y-%m-%d').dt.year

In [3]:
for_sale_df = realtor_data_df[(realtor_data_df["prev_sold_year"] > 2015.0) &
                                     (realtor_data_df["state"].isin(["New York"])) &
                                     (realtor_data_df["status"] == "for_sale") 
                                    ]
for_sale_df.drop(columns=["prev_sold_year"],inplace=True)
for_sale_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12916 entries, 1 to 67140
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      12916 non-null  int64  
 1   status          12916 non-null  object 
 2   bed             11715 non-null  float64
 3   bath            12165 non-null  float64
 4   acre_lot        8887 non-null   float64
 5   city            12916 non-null  object 
 6   state           12916 non-null  object 
 7   zip_code        12913 non-null  float64
 8   house_size      9165 non-null   float64
 9   prev_sold_date  12916 non-null  object 
 10  price           12916 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.2+ MB


In [4]:
zipcodes = for_sale_df["zip_code"]
len(zipcodes.unique())

1153

In [5]:
# Because of its size this dataset is not saved in the code repo for this project.
# It can be downloaded and saved in the same path to make this Notebook run.
zip_lon_lat = pd.read_csv('../../../zip_lat_long.csv')
zip_lon_lat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33144 entries, 0 to 33143
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ZIP     33144 non-null  int64  
 1   LAT     33144 non-null  float64
 2   LNG     33144 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 776.9 KB


In [6]:
find_zipcodes_df = zip_lon_lat[zip_lon_lat["ZIP"].isin(zipcodes.unique())]
find_zipcodes_df["zip_code_str"] = find_zipcodes_df.ZIP.apply(str)
find_zipcodes_df.head()

,ZIP,LAT,LNG,zip_code_str
2569,10001,40.750633,-73.997177,10001
2570,10002,40.715775,-73.986212,10002
2571,10003,40.731829,-73.989181,10003
2572,10004,40.688630,-74.018244,10004
2573,10005,40.706027,-74.008835,10005


In [7]:
uniquezip_df = pd.DataFrame(zipcodes.unique())
uniquezip_df.rename(columns={0: "ZIP"},inplace=True)
uniquezip_df["zip_code_str"] = uniquezip_df.ZIP.apply(str).str.slice(0,5)
uniquezip_df.head()

,ZIP,zip_code_str
0,12521.0,12521
1,12546.0,12546
2,12060.0,12060
3,12529.0,12529
4,12029.0,12029


merged_df.to_csv('zip-lon-lat.csv')

uniquezip.sort_values(0).to_csv("zipcodes.csv")

In [8]:
merge_df = uniquezip_df.merge(find_zipcodes_df, how='outer', on='zip_code_str')

In [9]:
zipnotfound_df = merge_df[merge_df.isna().any(axis=1)]
zipnotfound_df

,ZIP_x,zip_code_str,ZIP_y,LAT,LNG
129,10129.0,10129,NaN,NaN,NaN
176,11249.0,11249,NaN,NaN,NaN
205,11243.0,11243,NaN,NaN,NaN
288,10982.0,10982,NaN,NaN,NaN
621,NaN,nan,NaN,NaN,NaN


In [10]:
match_zipcodes = find_zipcodes_df.merge(for_sale_df[['zip_code',"city"]], how='outer', left_on='ZIP', right_on='zip_code')
match_zipcodes[match_zipcodes.isna().any(axis=1)]

,ZIP,LAT,LNG,zip_code_str,zip_code,city
12899,NaN,NaN,NaN,NaN,10129.0,New York
12900,NaN,NaN,NaN,NaN,11249.0,Brooklyn
12901,NaN,NaN,NaN,NaN,11249.0,New York City
12902,NaN,NaN,NaN,NaN,11249.0,New York City
12903,NaN,NaN,NaN,NaN,11249.0,Brooklyn
12904,NaN,NaN,NaN,NaN,11249.0,Brooklyn
12905,NaN,NaN,NaN,NaN,11249.0,Brooklyn
12906,NaN,NaN,NaN,NaN,11249.0,Brooklyn
12907,NaN,NaN,NaN,NaN,11249.0,Brooklyn
12908,NaN,NaN,NaN,NaN,11249.0,New York
